<a href="https://colab.research.google.com/github/Sharmila-KA/COVID-19-DATA-VISUALIZATION-AND-PREDICTION-/blob/main/projectml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import requests
import io
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from folium import plugins
import math
import random
from datetime import timedelta
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings 
warnings.filterwarnings('ignore')
#color pallette
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 12

In [2]:
import os
try:
    os.system("rm -rf Covid19Predictionproject")
except:
    print("File doesnot exsist")

In [3]:
df = pd.read_csv("/content/covid_19_data_cleaned.csv", parse_dates=['Date'])
country_daywise = pd.read_csv('/content/country_daywise.csv', parse_dates=['Date'])
countrywise = pd.read_csv('/content/countrywise.csv')
daywise = pd.read_csv('/content/daywise.csv', parse_dates=['Date'])
dd= pd.read_excel('/content/Covid cases in India.xlsx')
India_coord = pd.read_excel('/content/Indian Coordinates.xlsx')
dbd_India = pd.read_excel('/content/per_day_cases.xlsx',parse_dates=True, sheet_name='India')


df['Province/State'] = df['Province/State'].fillna("")
df.head()

,Date,Province/State,Country,Lat,Long,Confirmed,Recovered,Deaths,Active
0,2020-01-22,,Afghanistan,33.93911,67.709953,0,0,0,0
1,2020-01-23,,Afghanistan,33.93911,67.709953,0,0,0,0
2,2020-01-24,,Afghanistan,33.93911,67.709953,0,0,0,0
3,2020-01-25,,Afghanistan,33.93911,67.709953,0,0,0,0
4,2020-01-26,,Afghanistan,33.93911,67.709953,0,0,0,0


In [4]:

confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
deaths.head()


,Date,Deaths
0,2020-01-22,17
1,2020-01-23,18
2,2020-01-24,26
3,2020-01-25,42
4,2020-01-26,56


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = confirmed['Date'], y=confirmed['Confirmed'], mode = 'lines+markers', name = 'Confirmed', line = dict(color = "Orange", width = 4)))
fig.add_trace(go.Scatter(x = recovered['Date'], y=recovered['Recovered'], mode = 'lines+markers', name = 'Recovered', line = dict(color = "Green", width = 4)))
fig.add_trace(go.Scatter(x = deaths['Date'], y=deaths['Deaths'], mode = 'lines+markers', name = 'Deaths', line = dict(color = "Red", width = 4)))
fig.update_layout(title = 'Worldwide Covid-19 Cases', xaxis_tickfont_size = 14, yaxis=dict(title = 'No of cases'))
fig.show()


In [ ]:
df['Date'] = df['Date'].astype(str)
df.info()


In [ ]:
fig = px.density_mapbox(df, lat = 'Lat', lon = 'Long', hover_name = 'Country', hover_data = ['Confirmed' ,'Recovered', 'Deaths'], animation_frame='Date', color_continuous_scale='Portland', radius = 7,zoom = 0,height = 700)
fig.update_layout(title = 'Worldwide Covid-19 cases with timelapse')
fig.update_layout(mapbox_style = 'open-street-map', mapbox_center_lon = 0)
fig.show()

In [ ]:
temp =df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)
tm = temp.melt(id_vars = 'Date' , value_vars = ['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path = ['variable'], values = 'value', height = 250, width = 800,color_discrete_sequence=[act,rec,dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()


In [ ]:
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths','Active'], var_name = 'Case',value_name = 'Count')
fig = px.area(temp, x = 'Date', y = 'Count', color = 'Case', height = 600, title = 'Cases over time',color_discrete_sequence=[rec, dth, act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
fig = px.choropleth(country_daywise,locations = 'Country',locationmode = 'country names',color = country_daywise['Confirmed'],
                    hover_name = 'Country',animation_frame = country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                    title = 'Cases over time',color_continuous_scale = px.colors.sequential.Inferno)
fig.update(layout_coloraxis_showscale = True)
fig.show()

In [ ]:
fig_c = px.bar(daywise , x = 'Date' , y = 'Confirmed' , color_discrete_sequence=[act])
fig_d = px.bar(daywise , x = 'Date' , y = 'Deaths' , color_discrete_sequence=['aqua'])

fig = make_subplots(rows = 1,cols = 2, shared_xaxes=False, horizontal_spacing=0.1,
                   subplot_titles=("Confirmed Cases" , "Death Cases"))
fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_c['data'][0], row = 1, col = 2)
fig.update_layout(height = 400)
fig.show()

In [ ]:
fig_c = px.choropleth(countrywise,locations='Country',locationmode='country names',
                     color = np.log(countrywise['Confirmed']),hover_name='Country',
                     hover_data = ['Confirmed'])
temp = countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp,locations='Country',locationmode='country names',
                     color = np.log(temp['Deaths']),hover_name='Country',
                     hover_data = ['Deaths'])
fig = make_subplots(rows = 1,cols = 2,subplot_titles = ['Confirmed','Deaths'],
                    specs =[[{'type':'choropleth'},{'type':'choropleth'}]])

fig.add_trace(fig_c['data'][0],row = 1, col = 1)
fig.add_trace(fig_d['data'][0],row = 1, col = 2)
fig.show()
                  

In [ ]:
fig1 = px.line(daywise, x ='Date', y = 'Deaths / 100 Cases',color_discrete_sequence=[dth])
fig2 = px.line(daywise, x ='Date', y = 'Recovered / 100 Cases',color_discrete_sequence=[rec])
fig3 = px.line(daywise, x ='Date', y = 'Deaths / 100 Recovered',color_discrete_sequence=['aqua'])

fig = make_subplots(rows = 1,cols = 3, shared_xaxes= False,
                   subplot_titles=('Deaths / 100 Cases','Recovered / 100 Cases','Deaths / 100 Recovered'))
fig.add_trace(fig1['data'][0],row = 1, col = 1)
fig.add_trace(fig2['data'][0],row = 1, col = 2)
fig.add_trace(fig3['data'][0],row = 1, col = 3)

fig.update_layout(height = 400)
fig.show()

In [ ]:
fig_c = px.bar(daywise, x= 'Date', y = 'Confirmed', color_discrete_sequence=[cnf])
fig_d = px.bar(daywise, x= 'Date', y = 'No. of Countries', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1,cols = 3, shared_xaxes= False,horizontal_spacing=0.1,
                    subplot_titles=("No, of New Cases per Day","No. of Countries"))
fig.add_trace(fig_c['data'][0], row = 1,col = 1)
fig.add_trace(fig_d['data'][0], row = 1,col = 2)

fig.show()


In [ ]:
top = 15
fig_c = px.bar(countrywise.sort_values('Confirmed').tail(top),x='Confirmed',y='Country',
              text = 'Confirmed',orientation='h',color_discrete_sequence=[cnf])
fig_d = px.bar(countrywise.sort_values('Deaths').tail(top),x='Deaths',y='Country',
              text = 'Deaths',orientation='h',color_discrete_sequence=[dth])

fig_a = px.bar(countrywise.sort_values('Active').tail(top),x='Active',y='Country',
              text = 'Active',orientation='h',color_discrete_sequence=[act])
fig_r = px.bar(countrywise.sort_values('Recovered').tail(top),x='Recovered',y='Country',
              text = 'Recovered',orientation='h',color_discrete_sequence=[rec])
fig_dc = px.bar(countrywise.sort_values('Deaths / 100 Cases').tail(top),x='Deaths / 100 Cases',y='Country',
              text = 'Deaths / 100 Cases',orientation='h',color_discrete_sequence=['#f84351'])
fig_rc = px.bar(countrywise.sort_values('Recovered / 100 Cases').tail(top),x='Recovered / 100 Cases',y='Country',
              text = 'Recovered / 100 Cases',orientation='h',color_discrete_sequence=['#a45398'])




fig = make_subplots(rows = 5,cols = 2, shared_xaxes= False,horizontal_spacing=0.2,
                    vertical_spacing=0.05,
                    subplot_titles=('Confirmed Cases','Deaths Reported','Active Cases','Recovered Cases','Deaths / 100 Cases','Recovered / 100 Cases'))

fig.add_trace(fig_c['data'][0], row = 1,col = 1)
fig.add_trace(fig_d['data'][0], row = 1,col = 2)
fig.add_trace(fig_a['data'][0], row = 2,col = 1)
fig.add_trace(fig_r['data'][0], row = 2,col = 2)
fig.add_trace(fig_dc['data'][0], row = 3,col = 1)
fig.add_trace(fig_rc['data'][0], row = 3,col = 2)
fig.update_layout(height=4000)

fig.show()



In [ ]:
dd.style.background_gradient(cmap='Reds')

In [ ]:
dd.drop(['S.No.'],axis=1,inplace=True,errors='ignore')
dd['Total cases']=dd['Confirmed_i']+dd['Total Confirmed cases ( Foreign National )']
total_cases=dd['Total cases'].sum()
print('The total number of cases till now in India is',total_cases)

In [ ]:
dd['Total Active'] = dd['Total cases'] - (dd['Death'] + dd['Cured'])
total_active = dd['Total Active'].sum()
print('Total number of active COVID 2019 cases across India:', total_active)
Tot_Cases = dd.groupby('Name of State / UT')['Total Active'].sum().sort_values(ascending=False).to_frame()
Tot_Cases.style.background_gradient(cmap='Reds')

In [ ]:
dd_full = pd.merge(India_coord,dd,on='Name of State / UT')
map = folium.Map(location=[20, 70], zoom_start=4,tiles='Stamenterrain')

for lat, lon, value, name in zip(dd_full['Latitude'], dd_full['Longitude'], dd_full['Total cases'], dd_full['Name of State / UT']):
    folium.CircleMarker([lat, lon], radius=value*0.8, popup = ('<strong>State</strong>: ' + str(name).capitalize() + '<br>''<strong>Total Cases</strong>: ' + str(value) + '<br>'),color='red',fill_color='red',fill_opacity=0.3 ).add_to(map)
map


In [ ]:
sns.heatmap(daywise.corr(), cmap="YlGnBu", annot = True)
plt.show()

In [ ]:
df.rename(columns={'ObservationDate': 'Date',
                          'Country/Region': 'Country'}, inplace=True)


In [ ]:
df = df.groupby(["Date", "Country"])[
    ['Date', 'Country', 'Confirmed', 'Deaths', 'Recovered']].sum().reset_index()
corona_sorted_df = df.sort_values('Date', ascending=False)
corona_sorted_df = corona_sorted_df.drop_duplicates('Country')

In [ ]:
corona_confirmed_total_df = corona_sorted_df['Confirmed'].sum()
corona_deaths_total_df = corona_sorted_df['Deaths'].sum()
corona_recovered_total_df = corona_sorted_df['Recovered'].sum()

corona_confirmed_total_df = round(corona_confirmed_total_df, 2)
corona_deaths_total_df = round(corona_deaths_total_df, 2)
corona_recovered_total_df = round(corona_recovered_total_df, 2)

corona_dict = {'Total Confirmed cases  in the world': corona_confirmed_total_df,
               'Total Deaths cases in the world': corona_deaths_total_df, 'Total Recovered cases in the world': corona_recovered_total_df}
corona_dict = pd.DataFrame.from_dict(
    corona_dict, orient='index', columns=['Total'])

corona_dict.style.background_gradient(cmap='Blues')

In [ ]:
corona_dict = corona_dict.head(3)
x = corona_dict.index
y = corona_dict['Total'].values

plt.rcParams['figure.figsize'] = (10, 6)
plt.bar(x, y, color=['blue', 'red', 'green'])
plt.title('Total Confirmed / Deaths / Recovered cases worldwide(until 04/14)')
plt.show()

In [ ]:
sorted_By_Confirmed1 = corona_sorted_df.sort_values(
    'Confirmed', ascending=False)
sorted_By_Confirmed1 = sorted_By_Confirmed1.head(10)
x = sorted_By_Confirmed1.Country
y = sorted_By_Confirmed1.Confirmed
plt.rcParams['figure.figsize'] = (12, 10)
plt.bar(x, y, color=(0.2, 0.4, 0.6, 0.6))
plt.title('Top 10 infected countries(until 04/14)')
plt.show()

In [ ]:
corona_cases_per_day = df.groupby(
    ["Date"])['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()
sorted_By_Confirmed1 = corona_cases_per_day.sort_values(
    'Date', ascending=False)

sorted_By_Confirmed1.style.background_gradient(cmap='Reds')

In [ ]:
x = corona_cases_per_day.Date
y = corona_cases_per_day.Confirmed
y1 = corona_cases_per_day.Deaths
y2 = corona_cases_per_day.Recovered
ax = plt.gca()
corona_cases_per_day.plot(kind='line', x='Date',
                          y='Confirmed', color='blue', ax=ax)
corona_cases_per_day.plot(kind='line', x='Date',
                          y='Deaths', color='red', ax=ax)
corona_cases_per_day.plot(kind='line', x='Date',
                          y='Recovered', color='green', ax=ax)
print("Blue : Confirmed Cases ")
print("Red : Death Cases ")
print("Green : Recovered Cases ")
plt.ylabel('Total cases(#)')
plt.title('Total confirmed, death and recovered case report per day.')
plt.show()

# Linear regression india

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataset = pd.read_excel('Desktop\miniprojects\Covid-19-Preprocessed-Dataset\Sample Datasets/Covid cases in India.xlsx')
dataset.head()

In [ ]:
X = dataset['Confirmed_i']
Y = dataset['Cured']
plt.scatter(X,Y);
plt.xlabel('Confirmed Cases')
plt.ylabel('Cured')
plt.title('Confirmed vs Cured')
plt.show


In [ ]:
X = dataset['Confirmed_i']
Y = dataset['Cured']
# mean of our inputs and outputs
x_mean = np.mean(X)
y_mean = np.mean(Y)
#total number of values
n = len(X)
# using the formula to calculate the b1 and b0
numerator = 0
denominator = 0
for i in range(n):
    numerator += (X[i] - x_mean) * (Y[i] - y_mean)
    denominator += (X[i] - x_mean) ** 2
    
b1 = numerator / denominator
b0 = y_mean - (b1 * x_mean)
#printing the coefficient
print('b0 value :')
print(b0)
print('')
print('b1 value :')
print(b1)


In [ ]:
x = float(input('Input the no of Confirmed Cases : '))
y = b0 + float(b1 * x)
print('')
print('')
print("The predicted no of cured :")
print(round(y))


In [ ]:
x_max = np.max(X) + 100
x_min = np.min(X) + 0
#calculating line values of x and y
x = np.linspace(x_min, x_max, 1000)
y = b0 + b1 * x
#plotting line 
plt.plot(x, y, color='red', label='Linear Regression')
#plot the data point
plt.scatter(X, Y, color='blue', label='Data Point')
# x-axis label
plt.xlabel('Confirmed')
#y-axis label
plt.ylabel('Cured')
plt.legend()
plt.show()

In [ ]:
rmse = 0
for i in range(n):
    y_pred=  b0 + b1* X[i]
    rmse += (Y[i] - y_pred) ** 2
    
rmse = np.sqrt(rmse/n)
print('RMSE :')
print(rmse)

sumofsquares = 0
sumofresiduals = 0
for i in range(n) :
    y_pred = b0 + b1 * X[i]
    sumofsquares += (Y[i] - y_mean) ** 2
    sumofresiduals += (Y[i] - y_pred) **2
    
score  = 1 - (sumofresiduals/sumofsquares)
print('')
print('')
print('Sum of Squares :')
print(score)

# Confirmed,death linear regression in india

In [ ]:
X = dataset['Confirmed_i']
Y = dataset['Death']
plt.scatter(X,Y);
plt.xlabel('Confirmed Cases')
plt.ylabel('Death')
plt.title('Confirmed vs Death')
plt.show


In [ ]:
X = dataset['Confirmed_i']
Y = dataset['Death']
# mean of our inputs and outputs
x_mean = np.mean(X)
y_mean = np.mean(Y)
#total number of values
n = len(X)
# using the formula to calculate the b1 and b0
numerator = 0
denominator = 0
for i in range(n):
    numerator += (X[i] - x_mean) * (Y[i] - y_mean)
    denominator += (X[i] - x_mean) ** 2
    
b1 = numerator / denominator
b0 = y_mean - (b1 * x_mean)
#printing the coefficient
print('b0 value :')
print(b0)
print('')
print('b1 value :')
print(b1)

In [ ]:
x = float(input('Input the no of Confirmed Cases : '))
y = b0 + float(b1 * x)
print('')
print('')
print("The predicted no of Death :")
print(round(y))

In [ ]:
x_max = np.max(X) + 100
x_min = np.min(X) + 0
#calculating line values of x and y
x = np.linspace(x_min, x_max, 1000)
y = b0 + b1 * x
#plotting line 
plt.plot(x, y, color='red', label='Linear Regression')
#plot the data point
plt.scatter(X, Y, color='blue', label='Data Point')
# x-axis label
plt.xlabel('Confirmed')
#y-axis label
plt.ylabel('Cured')
plt.legend()
plt.show()

In [ ]:
rmse = 0
for i in range(n):
    y_pred=  b0 + b1* X[i]
    rmse += (Y[i] - y_pred) ** 2
    
rmse = np.sqrt(rmse/n)
print('RMSE :')
print(rmse)

sumofsquares = 0
sumofresiduals = 0
for i in range(n) :
    y_pred = b0 + b1 * X[i]
    sumofsquares += (Y[i] - y_mean) ** 2
    sumofresiduals += (Y[i] - y_pred) **2
    
score  = 1 - (sumofresiduals/sumofsquares)
print('')
print('')
print('Sum of Squares :')
print(score)

# Confirmed vs recovered linear regression with animation

In [ ]:
from numpy import array
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from subprocess import check_output
import networkx as nx
from matplotlib.animation import FuncAnimation, PillowWriter 


In [ ]:
import matplotlib.animation as animation
data = pd.read_csv('Desktop\miniprojects\Covid-19-Preprocessed-Dataset\Sample Datasets\daywise.csv')
x = data['Confirmed']
y = data['Recovered']

x = (x - x.mean()) / x.std()
x = np.c_[np.ones(x.shape[0]), x] 

In [ ]:
alpha = 0.01 #Step size
iterations = 2000 
m = y.size 
np.random.seed(123) 
theta = np.random.rand(2) 
def gradient_descent(x, y, theta, iterations, alpha):
    past_costs = []
    past_thetas = [theta]
    for i in range(iterations):
        prediction = np.dot(x, theta)
        error = prediction - y
        cost = 1/(2*m) * np.dot(error.T, error)
        past_costs.append(cost)
        theta = theta - (alpha * (1/m) * np.dot(x.T, error))
        past_thetas.append(theta)
        
    return past_thetas, past_costs
past_thetas, past_costs = gradient_descent(x, y, theta, iterations, alpha)
theta = past_thetas[-1]
print("Gradient Descent: {:.2f}, {:.2f}".format(theta[0], theta[1]))

In [ ]:
plt.title('Cases')
plt.xlabel('Confirmed')
plt.ylabel('Recovered')
plt.plot(past_costs)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.title('Cases')
plt.xlabel('Confirmed (normalised)')
plt.ylabel('Recovered ')
plt.scatter(x[:,1], y, color='red')
line, = ax.plot([], [], lw=2)
annotation = ax.text(-1, 700000, '')
annotation.set_animated(True)
plt.close()

#Generate the animation data,
def init():
    line.set_data([], [])
    annotation.set_text('')
    return line, annotation

# animation function.  This is called sequentially
def animate(i):
    x = np.linspace(-5, 20, 1000)
    y = past_thetas[i][1]*x + past_thetas[i][0]
    line.set_data(x, y)
    annotation.set_text('Cost = %.2f e10' % (past_costs[i]/10000000000))
    return line, annotation

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=300, interval=0, blit=True)

anim.save('animation.gif', writer='imagemagick', fps = 30)

In [ ]:
import io
import base64
from IPython.display import HTML

filename = 'animation.gif'

video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))